In this notebook we compute two features for the movielens ratings dataset. Each row of the dataset consists of the rating a user gave to certain movie. 

These features are roughly defined as follows:

**nb_previous_ratings:** number of ratings a user has given, prior to the current rating. 

**avg_ratings_previous:** average of the ratings a user has given, prior to the current rating.

We have the following problem: the ratings dataset is big (20M rows), and the feature `avg_ratings_previous` takes just too long to be computed for all the rows of the dataset (at least the way it's implemented). 

We have truncated the dataset so that this code runs quickly. 

We begin by extracting and truncating the dataset

In [25]:
import pandas as pd

In [26]:
df_ratings = pd.read_csv("rating.csv")

In [27]:
df_ratings

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [34]:
# Here we truncate the dataset
df_ratings_truncated = df_ratings.loc[0:10000, :]

In [35]:
#df_ratings_truncated = df_ratings

We now proceed to compute the first feature: `nb_previous_ratings`

In [36]:
df_ratings_truncated = df_ratings_truncated.sort_values(by=["userId", "timestamp"])
df_ratings_truncated = df_ratings_truncated.reset_index(drop=True)
df_ratings_truncated["timestamp"] = pd.to_datetime(df_ratings_truncated.timestamp)
df_ratings_truncated

,userId,movieId,rating,timestamp
0,1,924,3.5,2004-09-10 03:06:38
1,1,919,3.5,2004-09-10 03:07:01
2,1,2683,3.5,2004-09-10 03:07:30
3,1,1584,3.5,2004-09-10 03:07:36
4,1,1079,4.0,2004-09-10 03:07:45
...,...,...,...,...
9996,91,1188,3.0,2005-09-12 07:00:49
9997,91,2150,3.0,2005-09-12 07:00:52
9998,91,237,2.5,2005-09-12 07:01:12
9999,91,2490,3.5,2005-09-12 07:01:17


In [37]:
df_grouped = df_ratings_truncated.groupby("userId")
df_ratings_truncated["nb_previous_ratings"] = df_grouped["timestamp"].rank(method="first") - 1

In [38]:
df_ratings_truncated

,userId,movieId,rating,timestamp,nb_previous_ratings
0,1,924,3.5,2004-09-10 03:06:38,0.0
1,1,919,3.5,2004-09-10 03:07:01,1.0
2,1,2683,3.5,2004-09-10 03:07:30,2.0
3,1,1584,3.5,2004-09-10 03:07:36,3.0
4,1,1079,4.0,2004-09-10 03:07:45,4.0
...,...,...,...,...,...
9996,91,1188,3.0,2005-09-12 07:00:49,375.0
9997,91,2150,3.0,2005-09-12 07:00:52,376.0
9998,91,237,2.5,2005-09-12 07:01:12,377.0
9999,91,2490,3.5,2005-09-12 07:01:17,378.0


We now compute the second feature: `avg_ratings_previous`

In [39]:
def avg_previous(df):
    avg = pd.Series(index=df.index, dtype='float')
    for i in df.index:
        df_aux = df.loc[df.timestamp < df.timestamp.loc[i], :]
        avg.at[i] = df_aux.rating.mean()
    return avg

In [40]:
avg_ratings_previous = pd.Series()
# the following cycle is the one that takes forever if we try to compute it for the whole dataset
for user in df_ratings_truncated.userId.unique():
    df_user = df_ratings_truncated.loc[df_ratings_truncated.userId == user, :]
    avg_ratings_previous = avg_ratings_previous.append(avg_previous(df_user))
avg_ratings_previous

C:\Users\kingsun\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


0             NaN
1        3.500000
2        3.500000
3        3.500000
4        3.500000
           ...   
9996     3.376000
9997     3.375000
9998     3.374005
9999     3.371693
10000    3.372032
Length: 10001, dtype: float64

In [41]:
df_ratings_truncated["avg_ratings_previous"] = avg_ratings_previous
df_ratings_truncated

,userId,movieId,rating,timestamp,nb_previous_ratings,avg_ratings_previous
0,1,924,3.5,2004-09-10 03:06:38,0.0,NaN
1,1,919,3.5,2004-09-10 03:07:01,1.0,3.500000
2,1,2683,3.5,2004-09-10 03:07:30,2.0,3.500000
3,1,1584,3.5,2004-09-10 03:07:36,3.0,3.500000
4,1,1079,4.0,2004-09-10 03:07:45,4.0,3.500000
...,...,...,...,...,...,...
9996,91,1188,3.0,2005-09-12 07:00:49,375.0,3.376000
9997,91,2150,3.0,2005-09-12 07:00:52,376.0,3.375000
9998,91,237,2.5,2005-09-12 07:01:12,377.0,3.374005
9999,91,2490,3.5,2005-09-12 07:01:17,378.0,3.371693


In [47]:
df_ratings_truncated.loc[df_ratings_truncated.userId == 1, ]

,userId,movieId,rating,timestamp,nb_previous_ratings,avg_ratings_previous
0,1,924,3.5,2004-09-10 03:06:38,0.0,NaN
1,1,919,3.5,2004-09-10 03:07:01,1.0,3.500000
2,1,2683,3.5,2004-09-10 03:07:30,2.0,3.500000
3,1,1584,3.5,2004-09-10 03:07:36,3.0,3.500000
4,1,1079,4.0,2004-09-10 03:07:45,4.0,3.500000
...,...,...,...,...,...,...
170,1,5999,3.5,2005-04-02 23:55:50,170.0,3.757396
171,1,7449,3.5,2005-04-02 23:56:03,171.0,3.751462
172,1,4133,3.0,2005-04-02 23:56:09,172.0,3.750000
173,1,3997,3.5,2005-04-02 23:56:32,173.0,3.745665
